In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install nltk
!pip install huggingface_hub

In [ ]:
from nltk import download
download('punkt')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Preprocess data

In [ ]:
import json
import random

from nltk.tokenize import word_tokenize

def get_json(json_path: str):
    with open(json_path) as json_file:
        return json.load(json_file)


def shuffle(input_json):
    result = []
    for docset in input_json:
        input_texts = docset["text"]
        for input_text in input_texts:
            input_text_result = []
            total_token = 3
            for count, value in enumerate(input_text):
                token_length = len(word_tokenize(value))
                if total_token + token_length + 1 < 800:
                    input_text_result.append((count, value))
                    total_token = total_token + token_length + 1
                else:
                    break
            random.shuffle(input_text_result)
            input_text_result = [list(t) for t in zip(*input_text_result)]
            result.append(input_text_result)
    return result


def combine_data(json_path: str, dataset_type: str):
    input_json = get_json(json_path)
    examples = []
    for docset in input_json[dataset_type]:
        input_texts = docset["text"]
        examples.append(
            (
                f"[shuffled] {' '.join([' '.join((f'<S{i}>', sent)) for i, sent in zip(list(range(len(input_texts))), input_texts)])} [orig]"
            )
        )
    return examples


def load_data(json_path: str):
    """
    Loads the dataset file:
    json_path: json file
    Returns a list of tuples (input, output)
    """
    input_json = get_json(json_path)
    all_lines = shuffle(input_json)
    examples = []
    for line in all_lines:
        if len(line) == 2:
            examples.append(
                (
                    f"[shuffled] {' '.join([' '.join((f'<S{i}>', sent)) for i, sent in zip(list(range(len(line[0]))), line[1])])} [orig]",
                    f"{' '.join([str(j) for j in line[0]])} <eos>",
                )
            )
    return examples

Necessary Helper Function

In [ ]:
import os
import re
import glob
import torch
import random
import shutil
import logging
import numpy as np

from tqdm import tqdm, trange
from torch.nn import CrossEntropyLoss
from transformers import AdamW, get_linear_schedule_with_warmup, AutoModelWithLMHead, AutoTokenizer
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.tensorboard import SummaryWriter

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.DEBUG,
)
logger = logging.getLogger(__name__)


def init_model(model_name: str, device, do_lower_case: bool = False, args=None):
    """
    Initialize a pre-trained LM
    :param model_name: from MODEL_CLASSES
    :param device: CUDA / CPU device
    :param do_lower_case: whether the model is lower cased or not
    :return: the model and tokenizer
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=do_lower_case, use_fast=False)
    model = AutoModelWithLMHead.from_pretrained(model_name)
    model.to(device)
    model.eval()
    return tokenizer, model


def set_seed(args):
    """
    Set the random seed for reproducibility
    """
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)


def get_loss(args, batch, model):
    """
    Compute this batch loss
    """
    token_ids = batch["inputs"].to(args.device)
    input_mask = batch["input_mask"].to(args.device)

    # We don't send labels to model.forward because we want to compute per token loss
    lm_logits = model(token_ids, attention_mask=input_mask)[0]
    shift_logits = lm_logits[..., :-1, :].contiguous()
    batch_size, max_length, vocab_size = shift_logits.shape

    # Compute loss for each instance and each token
    loss_fct = CrossEntropyLoss(reduction="none")
    shift_logits = shift_logits.view(-1, vocab_size)
    shift_labels = token_ids[..., 1:].contiguous().view(-1)
    loss = loss_fct(shift_logits, shift_labels).view(batch_size, max_length)

    # Only consider non padded tokens
    loss_mask = input_mask[..., :-1].contiguous()
    loss = torch.mul(loss_mask, loss)  # [batch_size, max_length]

    return loss


def _rotate_checkpoints(args, checkpoint_prefix, use_mtime=False):
    """
    Keep a maximum of args.save_total_limit checkpoints.
    """
    if not args.save_total_limit:
        return

    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    glob_checkpoints = glob.glob(
        os.path.join(args.out_dir, "{}-*".format(checkpoint_prefix))
    )
    if len(glob_checkpoints) <= args.save_total_limit:
        return

    ordering_and_checkpoint_path = []
    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append(
                    (int(regex_match.groups()[0]), path)
                )

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    number_of_checkpoints_to_delete = max(
        0, len(checkpoints_sorted) - args.save_total_limit
    )
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info(
            "Deleting older checkpoint [{}] due to args.save_total_limit".format(
                checkpoint
            )
        )
        shutil.rmtree(checkpoint)


def train(args, train_dataset, model, tokenizer, loss_fnc=get_loss, eval_dataset=None):
    """
    Train the model.
    """
    tb_writer = SummaryWriter()
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size
    )

    # Set the number of steps based on the num_epochs * len(train) or args.max_steps if specified.
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = (
                args.max_steps
                // (len(train_dataloader) // args.gradient_accumulation_steps)
                + 1
        )
    else:
        t_total = (
                len(train_dataloader)
                // args.gradient_accumulation_steps
                * args.num_train_epochs
        )

    # Prepare optimizer and scheduler (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    optimizer = AdamW(
        optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon
    )
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist and load from there
    if os.path.isfile(
            os.path.join(args.model_name_or_path, "optimizer.pt")
    ) and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt")):
        optimizer.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "optimizer.pt"))
        )
        scheduler.load_state_dict(
            torch.load(os.path.join(args.model_name_or_path, "scheduler.pt"))
        )

    # Train
    total_batch_size = args.train_batch_size * args.gradient_accumulation_steps
    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {args.num_train_epochs}")
    logger.info(f"  Instantaneous batch size = {args.train_batch_size}")
    logger.info(
        f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}"
    )
    logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {t_total}")

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0

    # Check if continuing training from a checkpoint
    if os.path.exists(args.model_name_or_path):
        try:
            # set global_step to global_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (
                    len(train_dataloader) // args.gradient_accumulation_steps
            )
            steps_trained_in_current_epoch = global_step % (
                    len(train_dataloader) // args.gradient_accumulation_steps
            )

            logger.info(
                "  Continuing training from checkpoint, will skip to saved global_step"
            )
            logger.info(f"  Continuing training from epoch {epochs_trained}")
            logger.info(f"  Continuing training from global step {global_step}")
            logger.info(
                f"  Will skip the first {steps_trained_in_current_epoch} steps in the first epoch"
            )
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model_to_resize = model.module if hasattr(model, "module") else model
    model_to_resize.resize_token_embeddings(len(tokenizer))

    model.zero_grad()
    train_iterator = trange(epochs_trained, int(args.num_train_epochs), desc="Epoch")
    set_seed(args)  # Added here for reproducibility

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()

            # Take the loss only for the part after the input (as in seq2seq architecture)
            loss = loss_fnc(args, batch, model)
            loss = loss.mean()

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if args.eval_during_train:
                        results = evaluate(eval_dataset, args, model, loss_fnc=loss_fnc)
                        for key, value in results.items():
                            tb_writer.add_scalar(
                                "eval_{}".format(key), value, global_step
                            )
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar(
                        "loss",
                        (tr_loss - logging_loss) / args.logging_steps,
                        global_step,
                    )
                    logging_loss = tr_loss

                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"

                    # Save model checkpoint
                    out_dir = os.path.join(
                        args.out_dir, "{}-{}".format(checkpoint_prefix, global_step)
                    )

                    if not os.path.exists(out_dir):
                        os.makedirs(out_dir)

                    model_to_save = model.module if hasattr(model, "module") else model
                    model_to_save.save_pretrained(out_dir)
                    tokenizer.save_pretrained(out_dir)
                    torch.save(args, os.path.join(out_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", out_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(
                        optimizer.state_dict(), os.path.join(out_dir, "optimizer.pt")
                    )
                    torch.save(
                        scheduler.state_dict(), os.path.join(out_dir, "scheduler.pt")
                    )
                    logger.info("Saving optimizer and scheduler states to %s", out_dir)

            if 0 < args.max_steps < global_step:
                epoch_iterator.close()
                break

        if 0 < args.max_steps < global_step:
            train_iterator.close()
            break

    tb_writer.close()
    return global_step, tr_loss / global_step


def evaluate(eval_dataset, args, model, prefix="", loss_fnc=get_loss):
    """
    Evaluation
    """
    eval_out_dir = args.out_dir

    if not os.path.exists(eval_out_dir):
        os.makedirs(eval_out_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size
    )

    logger.info(f"***** Running evaluation {prefix} *****")
    logger.info(f"  Num examples = {len(eval_dataset)}")
    logger.info(f"  Batch size = {args.eval_batch_size}")
    micro_loss = macro_loss = 0.0
    num_tokens = num_batches = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        with torch.no_grad():
            batch_loss = loss_fnc(args, batch, model)
            macro_loss += batch_loss.mean().item()
            micro_loss += batch_loss.sum().item()
            num_tokens += batch_loss.view(-1).shape[0]
        num_batches += 1

    macro_perplexity = torch.exp(torch.tensor(macro_loss / num_batches))
    micro_perplexity = torch.exp(torch.tensor(micro_loss / num_tokens))

    result = {
        "macro_perplexity": macro_perplexity,
        "micro_perplexity": micro_perplexity,
    }

    output_eval_file = os.path.join(eval_out_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info(f"***** Eval results {prefix} *****")
        for key in sorted(result.keys()):
            logger.info(f"  {key} = {result[key]}")
            writer.write(f"{key} = {result[key]}\n")

    return result

In [ ]:
class EncoderDecoderTextDataset(Dataset):
    def __init__(self, tokenizer, args, file_path, block_size=512):
        print(file_path)
        assert os.path.isfile(file_path)
        directory, filename = os.path.split(file_path)
        filename = f"{os.path.basename(args.model_type)}_cached_{block_size}_{filename}{'_' + args.task if args.task else ''}"
        cached_features_file = os.path.join(directory, filename)

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info(f"Loading features from cached file {cached_features_file}")
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Converting to token IDs")
            examples = load_data(file_path)
            logger.info(examples[:5])

            # Add prefix to the output so we can predict the first real token in the decoder
            inputs = [
                tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ex[0]))
                for ex in examples
            ]
            outputs = [
                [inputs[i][-1]]
                + tokenizer.convert_tokens_to_ids(tokenizer.tokenize(ex[1]))
                for i, ex in enumerate(examples)
            ]

            # Pad
            max_input_length = min(
                args.max_input_length, max([len(ex) for ex in inputs])
            )
            max_output_length = min(
                args.max_output_length, max([len(ex) for ex in outputs])
            )

            input_lengths = [min(len(ex), max_input_length) for ex in inputs]
            output_lengths = [min(len(ex), max_output_length) for ex in outputs]

            inputs = [tokenizer.encode(
                ex, add_special_tokens=False, max_length=max_input_length, pad_to_max_length=True)
                for ex in inputs]

            outputs = [tokenizer.encode(
                ex, add_special_tokens=False, max_length=max_output_length, pad_to_max_length=True)
                for ex in outputs]

            self.examples = {
                "inputs": inputs,
                "outputs": outputs,
                "input_lengths": input_lengths,
                "output_lengths": output_lengths,
            }

        logger.info(f"Saving features into cached file {cached_features_file}")
        with open(cached_features_file, "wb") as handle:
            pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples["input_lengths"])

    def __getitem__(self, item):
        inputs = torch.tensor(self.examples["inputs"][item])
        outputs = torch.tensor(self.examples["outputs"][item])

        max_length = inputs.shape[0]
        input_lengths = self.examples["input_lengths"][item]
        input_mask = torch.tensor([1] * input_lengths + [0] * (max_length - input_lengths))

        max_length = outputs.shape[0]
        output_lengths = self.examples["output_lengths"][item]
        output_mask = torch.tensor([1] * output_lengths + [0] * (max_length - output_lengths))

        return {
            "inputs": inputs,
            "input_mask": input_mask,
            "outputs": outputs,
            "output_mask": output_mask,
        }

Start of Main Pipeline

In [ ]:
import os
import torch
import pickle
import logging
import argparse

from torch.utils.data import Dataset
from torch.nn import CrossEntropyLoss, DataParallel

parser = argparse.ArgumentParser()

# Required parameters
parser.add_argument(
    "--out_dir",
    default="/content/output",
    type=str,
    help="Out directory for checkpoints.",
)

# Other parameters
parser.add_argument(
    "--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer."
)
parser.add_argument(
    "--device", default="cpu", type=str, help="GPU number or 'cpu'."
)
parser.add_argument(
    "--do_eval", action="store_true", help="Whether to run eval on the dev set."
)
parser.add_argument(
    "--do_lower_case",
    action="store_true",
    help="Set this flag if you are using an uncased model.",
)
parser.add_argument(
    "--do_train", action="store_true", help="Whether to run training."
)
parser.add_argument(
    "--eval_batch_size", default=8, type=int, help="Batch size for evaluation."
)
parser.add_argument(
    "--eval_data_file",
    type=str,
    default="/content/devtest.json",
    help="The input CSV validation file."
)
parser.add_argument(
    "--eval_during_train",
    action="store_true",
    help="Evaluate at each train logging step.",
)
parser.add_argument(
    "--gradient_accumulation_steps",
    type=int,
    default=1,
    help="Steps before backward pass.",
)
parser.add_argument(
    "--learning_rate",
    default=5e-6,
    type=float,
    help="The initial learning rate for Adam.",
)
parser.add_argument(
    "--logging_steps",
    type=int,
    default=-1,
    help="Log every X updates steps (default after each epoch).",
)
parser.add_argument(
    "--max_input_length",
    default=1024,
    type=int,
    help="Maximum input event length in words.",
)
parser.add_argument(
    "--max_output_length",
    default=120,
    type=int,
    help="Maximum output event length in words.",
)
parser.add_argument(
    "--max_grad_norm", default=1.0, type=float, help="Max gradient norm."
)
parser.add_argument(
    "--max_steps",
    default=-1,
    type=int,
    help="If > 0: total number of training steps to perform.",
)
parser.add_argument(
    "--model_name_or_path",
    default="facebook/bart-base",
    type=str,
    help="LM checkpoint for initialization.",
)
parser.add_argument(
    "--model_type",
    default="",
    type=str,
    help="which family of LM, e.g. gpt, gpt-xl, ....",
)
parser.add_argument(
    "--num_train_epochs",
    default=8.0,
    type=float,
    help="Number of training epochs to perform.",
)
parser.add_argument(
    "--overwrite_cache", action="store_true", help="Overwrite the cached data."
)
parser.add_argument(
    "--overwrite_out_dir",
    action="store_true",
    help="Overwrite the output directory.",
)
parser.add_argument(
    "--continue_training",
    action="store_true",
    help="Continue training from the last checkpoint.",
)
parser.add_argument(
    "--save_steps",
    type=int,
    default=-1,
    help="Save checkpoint every X updates steps (default after each epoch).",
)
parser.add_argument(
    "--save_total_limit",
    type=int,
    default=None,
    help="Maximum number of checkpoints to keep",
)
parser.add_argument(
    "--seed", type=int, default=43, help="Random seed for initialization."
)
parser.add_argument(
    "--train_batch_size", default=10, type=int, help="Batch size for training."
)
parser.add_argument(
    "--train_file",
    type=str,
    required=False,
    help="The input CSV train file."
)
parser.add_argument(
    "--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps."
)
parser.add_argument(
    "--weight_decay", default=0.0, type=float, help="Weight decay if we apply some."
)
parser.add_argument(
    "--task",
    type=str,
    help="what is the task?"
)
args, unknown = parser.parse_known_args()

In [ ]:
args.do_train = True
args.train_file = "/content/training.json"
args.do_eval = True
args.eval_during_train = True

In [ ]:
if args.eval_data_file is None and args.do_eval:
    raise ValueError(
        "Cannot do evaluation without an evaluation data file. Either supply --eval_data_file "
        "or remove the --do_eval argument."
    )

if (
    os.path.exists(args.out_dir)
    and len(os.listdir(args.out_dir)) > 1
    and args.do_train
    and not args.overwrite_out_dir
    and not args.continue_training
):
    raise ValueError(
        f"Output directory {args.out_dir} already exists and is not empty. "
        f"Use --overwrite_out_dir or --continue_training."
    )

In [ ]:
# Setup device
print(torch.cuda.is_available())
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

In [ ]:
# Set seed
set_seed(args)

# Load the models
if args.continue_training:
    args.model_name_or_path = args.out_dir
# Delete the current results file
else:
    eval_results_file = os.path.join(args.out_dir, "eval_results.txt")
    if os.path.exists(eval_results_file):
        os.remove(eval_results_file)

args.device = "cpu"
tokenizer, model = init_model(
    args.model_name_or_path, device=args.device, do_lower_case=args.do_lower_case, args = args
)

args.pad_token_id = tokenizer.pad_token_id
logger.info(f"Pad token ID: {args.pad_token_id}")
args.block_size = tokenizer.max_len_single_sentence
logger.info(f"Training/evaluation parameters {args}")

eval_dataset = None
if args.do_eval or args.eval_during_train:
    eval_dataset = EncoderDecoderTextDataset(
        tokenizer, args, file_path=args.eval_data_file, block_size=args.block_size)

# Add special tokens (if loading a model before fine-tuning)
if args.do_train and not args.continue_training:
    special_tokens = ["[shuffled]", "[orig]", "<eos>"]
    extra_specials = [f"<S{i}>" for i in range(args.max_output_length)]
    special_tokens += extra_specials
    tokenizer.pad_token = "<pad>"
    tokenizer.eos_token = "<eos>"
    tokenizer.add_tokens(special_tokens)
    model.resize_token_embeddings(len(tokenizer))

args.pad_token_id = tokenizer.pad_token_id

# resize_token_embeddings for Bart doesn't work if the model is already on the device
args.device = device
model= DataParallel(model)
model.to(args.device)

In [ ]:
# Training
if args.do_train:
    train_dataset = EncoderDecoderTextDataset(
        tokenizer,
        args,
        file_path=args.train_file,
        block_size=args.block_size,
    )
    global_step, tr_loss = train(
        args,
        train_dataset,
        model,
        tokenizer,
        loss_fnc=get_loss,
        eval_dataset=eval_dataset,
    )
    logger.info(f" global_step = {global_step}, average loss = {tr_loss}")

    # Create output directory if needed
    if not os.path.exists(args.out_dir):
        os.makedirs(args.out_dir)

    logger.info(f"Saving model checkpoint to {args.out_dir}")

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = model.module if hasattr(model, "module") else model
    model_to_save.save_pretrained(args.out_dir)
    tokenizer.save_pretrained(args.out_dir)

    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.out_dir, "training_args.bin"))

    # Load a trained model and vocabulary that you have fine-tuned
    tokenizer, model = init_model(
        args.out_dir, device=args.device, do_lower_case=args.do_lower_case, args=args
    )
    args.block_size = tokenizer.max_len_single_sentence
    model= DataParallel(model)
    model.to(args.device)

In [ ]:
# Evaluation
results = {}
if args.do_eval:
    checkpoint = args.out_dir
    logger.info(f"Evaluate the following checkpoint: {checkpoint}")
    prefix = (
        checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""
    )
    _, model = init_model(
        checkpoint, device=args.device, do_lower_case=args.do_lower_case, args=args
    )

    model.to(args.device)
    result = evaluate(eval_dataset, args, model, prefix=prefix, loss_fnc=get_loss)
    results.update(result)
print(results)

In [ ]:
def generate_conditional(tokenizer, model, args, input, device):
    """
    Generate a sequence with models like Bart and T5
    """
    input_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(input))
    decoder_start_token_id = input_ids[-1]
    input_ids = torch.tensor([input_ids]).to(device)
    max_length = args.max_length

    outputs = model.generate(
        input_ids,
        do_sample=args.beams == 0,
        max_length=max_length,
        min_length=5,
        temperature=args.temperature,
        top_p=args.p if args.p > 0 else None,
        top_k=args.k if args.k > 0 else None,
        num_beams=args.beams if args.beams > 0 else None,
        early_stopping=True,
        no_repeat_ngram_size=2,
        eos_token_id=tokenizer.eos_token_id,
        decoder_start_token_id=decoder_start_token_id,
        num_return_sequences=1  # max(1, args.beams)
    )

    preds = [tokenizer.decode(
        output, skip_special_tokens=False, clean_up_tokenization_spaces=False) for output in outputs]

    return preds

In [ ]:
import tqdm

"""
Generate outputs
"""
parser = argparse.ArgumentParser()

# Required
parser.add_argument(
    "--in_file",
    default="/content/dataset.json",
    type=str,
    help="The input json file",
)
parser.add_argument(
    "--out_file",
    default="/content/test.json",
    type=str,
    help="out jsonl file",
)
# Optional
parser.add_argument(
    "--max_length", default=1024, type=int, required=False, help="Maximum text length"
)
parser.add_argument(
    "--k", default=0, type=int, required=False, help="k for top k sampling"
)
parser.add_argument(
    "--p", default=0, type=float, required=False, help="p for nucleus sampling"
)
parser.add_argument(
    "--beams", default=1, type=int, required=False, help="beams for beam search"
)
parser.add_argument(
    "--temperature",
    default=1.0,
    type=float,
    required=False,
    help="temperature for sampling",
)
parser.add_argument(
    "--dataset_type",
    default="train",
    type=str,
    help="Which part of the dataset to load.",
)

args, unknown = parser.parse_known_args()

if (
        (args.k == args.p == args.beams == 0)
        or (args.k != 0 and args.p != 0)
        or (args.beams != 0 and args.p != 0)
        or (args.beams != 0 and args.k != 0)
):
    raise ValueError(
        "Exactly one of p, k, and beams should be set to a non-zero value."
    )

examples = combine_data(args.in_file, args.dataset_type)

logger.info(examples[:5])

special_tokens = ["[shuffled]", "[orig]", "<eos>"]
extra_specials = [f"<S{i}>" for i in range(args.max_length)]
special_tokens += extra_specials

with open(args.out_file, "w") as f_out:
    for input_lines in tqdm.tqdm(examples):
        try:
            preds = generate_conditional(
                tokenizer,
                model,
                args,
                input_lines,
                device,
            )

            # Remove any word that has "]" or "[" in it
            preds = [re.sub(r"(\w*\])", "", pred) for pred in preds]
            preds = [re.sub(r"(\[\w*)", "", pred) for pred in preds]
            preds = [re.sub(" +", " ", pred).strip() for pred in preds]

        except Exception as exp:
            logger.info(exp)
            preds = []

        f_out.write(
            json.dumps({"input": input_lines, "predictions": preds})
            + "\n"
        )

In [ ]:
model.push_to_hub("junyinc/LING575-WIN21-Reorder")
tokenizer.push_to_hub("junyinc/LING575-WIN21-Reorder")